In [35]:
from collections import defaultdict
file = open('input.txt', 'r')

def draw_to_dict(draw):
    draw_dict = defaultdict(int)
    for num, color in draw:
        draw_dict[color.strip()]=int(num)
    return draw_dict

games = {}
for line in file:
    game, sets = line.split(": ")
    game = int(game.split(" ")[1])
    sets = [s.split(', ') for s in sets.split("; ")]
    sets = [draw_to_dict([tuple(d.split(" ")) for d in s]) for s in sets]
    games[game]=sets
    
bag = {"red":12, "green":13, "blue":14}
print (bag)
print(games)

{'red': 12, 'green': 13, 'blue': 14}
{1: [defaultdict(<class 'int'>, {'blue': 1, 'green': 8}), defaultdict(<class 'int'>, {'green': 14, 'blue': 15}), defaultdict(<class 'int'>, {'green': 3, 'blue': 9}), defaultdict(<class 'int'>, {'green': 8, 'blue': 8, 'red': 1}), defaultdict(<class 'int'>, {'red': 1, 'green': 9, 'blue': 10})], 2: [defaultdict(<class 'int'>, {'blue': 3, 'green': 1, 'red': 2}), defaultdict(<class 'int'>, {'red': 2, 'green': 2, 'blue': 5}), defaultdict(<class 'int'>, {'green': 3, 'blue': 10}), defaultdict(<class 'int'>, {'red': 8, 'blue': 1}), defaultdict(<class 'int'>, {'red': 3, 'green': 1, 'blue': 5}), defaultdict(<class 'int'>, {'blue': 1, 'red': 5, 'green': 3})], 3: [defaultdict(<class 'int'>, {'green': 4, 'blue': 1}), defaultdict(<class 'int'>, {'blue': 6, 'green': 5, 'red': 1}), defaultdict(<class 'int'>, {'green': 11, 'blue': 10})], 4: [defaultdict(<class 'int'>, {'blue': 12, 'green': 12, 'red': 3}), defaultdict(<class 'int'>, {'blue': 15, 'green': 1, 'red': 10}

In [36]:
valid_games_sum = 0
for game in games:
    valid = True
    for g_set in games[game]:
        for color in bag:
            if bag[color] < g_set[color]:
                valid = False
                break
    if valid:
        valid_games_sum += game

{"part_1": valid_games_sum}




{'part_1': 2164}